# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

Nordvig's solution

In [98]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('data/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

phrase = "dking sport"
corrected_phrase = " ".join(correction(word) for word in phrase.split())
print(corrected_phrase)

king sport


In [99]:
import re
from collections import defaultdict

# Load N-gram counts from file
def load_ngrams(file_path, n=2):
    ngrams = defaultdict(int)
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip().split("\t")
            ngram = tuple(line[1:])
            count = int(line[0])
            ngrams[ngram] = count
    return ngrams

# Compute the probability of a given N-gram sequence
def ngram_prob(ngram, ngrams, n=2):
    count = ngrams[ngram]
    prefix = ngram[:-1]
    prefix_count = sum(ngrams[prefix_ngram] for prefix_ngram in ngrams if prefix_ngram[:-1] == prefix)
    return count / prefix_count if prefix_count > 0 else 0

# Generate possible correction candidates for a given misspelled word
def generate_candidates(word, vocab):
    candidates = []
    for i in range(len(word) + 1):
        for j in range(i + 1, len(word) + 2):
            prefix = word[:i]
            suffix = word[j:]
            for candidate in vocab:
                if candidate.startswith(prefix) and candidate.endswith(suffix):
                    candidates.append(prefix + candidate + suffix)
    return list(set(candidates))

# Score each correction candidate based on its probability in the given context
def score_candidates(sentence, candidates, ngrams, n=2):
    scored_candidates = []
    for candidate in candidates:
        words = sentence.split()
        for i, word in enumerate(words):
            if word == '<unk>':
                context = tuple(words[max(0, i - n + 1):i] + [candidate] + words[i + 1:i + n])
                prob = ngram_prob(context, ngrams, n)
                scored_candidates.append((candidate, prob))
    scored_candidates.sort(key=lambda x: x[1], reverse=True)
    return scored_candidates

# Spelling correction function
def correct_sentence(sentence, ngrams, n=2, vocab=None):
    if vocab is None:
        vocab = set(word for ngram in ngrams for word in ngram)
    words = sentence.split()
    corrected_words = []
    for word in words:
        if word not in vocab:
            candidates = generate_candidates(word, vocab)
            scored_candidates = score_candidates(' '.join(corrected_words + [word] + words[len(corrected_words) + 1:]), candidates, ngrams, n)
            if scored_candidates:
                corrected_word, _ = scored_candidates[0]
            else:
                corrected_word = word
        else:
            corrected_word = word
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

# Load N-gram counts
ngrams = load_ngrams('data/bigrams.txt')

# Example usage
sentence1 = "dking sport"
print(correct_sentence(sentence1, ngrams))


doing sport


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Justifications for the implementation choices in the given code:

1. **N-gram dataset choice**: The code uses a specific N-gram dataset loaded from the file "data/bigrams.txt". The choice of the dataset depends on the available data and the desired context for spelling correction.  I've tried different another dataset given, and see no accurasy increased, so i lefn only one dataset with bigrams

2. **N-gram Probability Calculation**: The code implements the `ngram_prob` function to compute the probability of a given N-gram sequence. It uses the count of the N-gram and the count of its prefix in the N-gram dataset to calculate the probability. The choice to use the count-based approach for probability calculation is a common practice in N-gram language modeling. By dividing the count of the N-gram by the count of its prefix, the code estimates the probability of the N-gram occurring in the given context. This approach assumes that the frequency of an N-gram in the dataset reflects its likelihood in the language.

3. **Candidate Generation**: The code implements the `generate_candidates` function to generate possible correction candidates for a given misspelled word. It generates candidates by inserting, deleting, or replacing characters in the misspelled word and checking if the resulting word is in the vocabulary. The choice to generate candidates through character-level manipulation is a common approach in spelling correction. By considering different combinations of characters, the code aims to find potential corrections that are similar to the misspelled word.

4. **Candidate Scoring**: The code implements the `score_candidates` function to score each correction candidate based on its probability in the given context using N-gram probabilities. It calculates the probability of each candidate by considering its context in the sentence and using the `ngram_prob` function. The choice to score candidates based on their context and N-gram probabilities is a common approach in spelling correction. By considering the surrounding words, the code aims to estimate the likelihood of each candidate being the correct correction for the misspelled word.

5. **Spelling Correction**: The code implements the `correct_sentence` function to perform spelling correction on a given sentence. It iterates through each word in the sentence and checks if it is in the vocabulary. If not, it generates correction candidates and scores them using the `generate_candidates` and `score_candidates` functions. The choice to correct misspelled words based on candidate scoring is a common approach in spelling correction. By selecting the candidate with the highest probability, the code aims to choose the most likely correction for each misspelled word.


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [67]:
import nltk
nltk.download('gutenberg')

from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\karin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


In [68]:

import random
import re

def corrupt_sentence(sentence, prob=0.5, mistake_type_prob=0.9):
    """
    Corrupts sentences by randomly introducing errors in words.
    The probability of introducing an error is controlled by the `prob` parameter.
    The probability of the error type (single edit or double edit) is controlled by the `mistake_type_prob` parameter.
    """    
    words = re.findall(r'\w+', sentence)
    for i, word in enumerate(words):
        if random.random() < prob and not word.isnumeric():
            words[i] = random.choice(list(edits1(word) if random.random() < mistake_type_prob else edits2(word)))
            if word.istitle():
                words[i] = words[i].capitalize()
    return re.sub(r'\w+', lambda m: words.pop(0), sentence)

sentence = "This is a test sentence."
corrupted_sentence = corrupt_sentence(sentence)
print(f"Original sentence: {sentence}")
print(f"Corrupted sentence: {corrupted_sentence}")


Original sentence: This is a test sentence.
Corrupted sentence: Thms is ap tost sentence.


In [106]:
#get test data
test_set = gutenberg.sents('austen-emma.txt')[100:200]

corrupted_test_set = []
for sentence in test_set:
    corrupted_sentence = corrupt_sentence(' '.join(sentence))
    corrupted_test_set.append(corrupted_sentence)


In [107]:
def evaluate_correction(corrected_sentence, correct_sentence):
    """
    Evaluates the correction of a corrupted sentence
    """
    correct_words = correct_sentence.split()
    corrupted_words = corrected_sentence.split()
    total_words = len(correct_words)
    correct_words_count = 0
    for i in range(total_words):
        if correct_words[i].lower() == corrupted_words[i].lower():
            correct_words_count += 1
    return correct_words_count / total_words



In [109]:
total_accuracy_nordvig = 0
total_accuracy_ngram = 0
count = 0

for correct_sentence, corrupted_sentence in zip(test_set, corrupted_test_set):
    nordvig_corrected_sentence = " ".join(correction(word) for word in corrupted_sentence.split())
    ngram_corrected_sentence = correct_sentence(corrupted_sentence, ngrams)
    accuracy_nordvig = evaluate_correction(nordvig_corrected_sentence, " ".join(correct_sentence))
    accuracy_ngram = evaluate_correction(ngram_corrected_sentence, " ".join(correct_sentence))

    total_accuracy_nordvig += accuracy_nordvig
    total_accuracy_ngram += accuracy_ngram

total_accuracy_ngram /= len(test_set)
total_accuracy_nordvig /= len(test_set)    


print(f"Average accuracy for Nordvig: {total_accuracy_nordvig}")
print(f"Average accuracy for N-gram: {total_accuracy_ngram}")

    

Average accuracy for Nordvig: 0.6874456277359771
Average accuracy for N-gram: 0.6620189110904304
